# AdventureWorks - Easy

![rel](https://sqlzoo.net/w/images/2/28/AdventureWorks.png)

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app14-1")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@18ac184a

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@7e37d1ee

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val cust_aw = hiveCxt.table("sqlzoo.CustomerAW")
val cust_addr = hiveCxt.table("sqlzoo.CustomerAddress")
val addr = hiveCxt.table("sqlzoo.Address")
val product = hiveCxt.table("sqlzoo.Product")
val order_det = hiveCxt.table("sqlzoo.SalesOrderDetail")
val order_head = hiveCxt.table("sqlzoo.SalesOrderHeader")

cust_aw: DataFrame = [customerid: int, namestyle: string ... 11 more fields]
cust_addr: DataFrame = [customerid: int, addressid: int ... 1 more field]
addr: DataFrame = [addressid: int, addressline1: string ... 5 more fields]
product: DataFrame = [productid: int, name: string ... 12 more fields]
order_det: DataFrame = [salesorderid: int, salesorderdetailid: int ... 4 more fields]
order_head: DataFrame = [salesorderid: int, revisionnumber: int ... 17 more fields]

## 1.
**Show the first name and the email address of customer with CompanyName "Bike World"**

In [5]:
(cust_aw.filter(col("CompanyName")==="Bike World")
 .select("FirstName", "EmailAddress")
 .showHTML())

FirstName,EmailAddress
Kerim,kerim0@adventure-works.com


## 2.
**Show the CompanyName for all customers with an address in City 'Dallas'.**

In [6]:
(cust_aw.join(cust_addr, "CustomerID")
 .join(addr.filter(col("City")==="Dallas"), "AddressID")
 .select("CompanyName")
 .distinct()
 .orderBy("CompanyName")
 .showHTML())

CompanyName
Elite Bikes
Rental Bikes
Third Bike Store
Town Industries
Unsurpassed Bikes


## 3.
**How many items with ListPrice more than $1000 have been sold?**

In [7]:
(order_det.join(product.filter(col("ListPrice")>1000), "ProductID")
 .groupBy()
 .agg(sum("OrderQty").alias("qty"))
 .showHTML())

qty
451


## 4.
**Give the CompanyName of those customers with orders over $100000. Include the subtotal plus tax plus freight.**

In [8]:
(order_head.join(cust_aw, "CustomerID")
 .withColumn("n_orders", col("SubTotal") + col("TaxAmt") + col("Freight"))
 .filter(col("n_orders")>100000)
 .select("CompanyName")
 .distinct()
 .showHTML())

CompanyName
Metropolitan Bicycle Supply
Action Bicycle Specialists


## 5.
**Find the number of left racing socks ('Racing Socks, L') ordered by CompanyName 'Riding Cycles'**

In [9]:
(order_det.join(product, "ProductID")
 .join(order_head, "SalesOrderID")
 .join(cust_aw, "CustomerID")
 .filter((col("Name")==="Racing Socks, L") && 
         (col("CompanyName")==="Riding Cycles"))
 .groupBy()
 .agg(sum("OrderQty").alias("total"))
 .showHTML())

total
3


In [10]:
spark.stop()